## Checking Special Functions
This notebook intends to explain the behavior of some special functions, which are used during the training

Import some important libraries

In [36]:
import sys
sys.path.append('../../')
import os
import tensorflow as tf
import functools
import json
import numpy as np
from learning_to_simulate import reading_utils
from learning_to_simulate import train
from learning_to_simulate import learned_simulator

tf.compat.v1.enable_eager_execution()

Define the function to read the metadata

In [2]:
def _read_metadata(data_path):
  with open(os.path.join(data_path, 'metadata.json'), 'rt') as fp:
    return json.loads(fp.read())

Load the input data using the `input_fn()` function

In [3]:
info_dir = "/home/zoso/Documents/deepmind-research/information"
data_path = os.path.join(info_dir,'datasets/WaterDropSample/')

batch_size = 2
mode = 'one_step_train'
split = 'train'

input_fn = train.get_input_fn(data_path, batch_size,mode=mode, split=split)

dataset = input_fn()

Now, we are going to use some special function

## Get the velocities from positions
    
$$\dot{p}^{t_k} = p^{t_k} - p^{t_{k-1}}  $$

`time_diff` just gets the difference between the next and previous positions

In [22]:
for features, labels in dataset.take(1):
    particle_type = features['particle_type'][:2]
    position_sequence = features['position'][:2]
    velocity_sequence = learned_simulator.time_diff(features['position'])[:2]
    n_particles_per_example = features['n_particles_per_example']
    print("Particle types: ")
    print(particle_type)
    print("Positions one particle: ")
    print(position_sequence)
    print("Velocities one particle: ")
    print(velocity_sequence)
    print("Num nodes per examples: ")
    print(n_particles_per_example)

Particle types: 
tf.Tensor([5 5], shape=(2,), dtype=int64)
Positions one particle: 
tf.Tensor(
[[[0.7872539  0.16003212]
  [0.78718716 0.15989359]
  [0.7871062  0.15974379]
  [0.7870073  0.1595886 ]
  [0.7868904  0.15943238]
  [0.7867547  0.15928067]]

 [[0.75501305 0.15070474]
  [0.75471425 0.15050365]
  [0.7543989  0.15029311]
  [0.7540701  0.15008163]
  [0.7537295  0.14987446]
  [0.7533809  0.14967577]]], shape=(2, 6, 2), dtype=float32)
Velocities one particle: 
tf.Tensor(
[[[-6.6757202e-05 -1.3853610e-04]
  [-8.0943108e-05 -1.4980137e-04]
  [-9.8943710e-05 -1.5518069e-04]
  [-1.1688471e-04 -1.5622377e-04]
  [-1.3566017e-04 -1.5170872e-04]]

 [[-2.9879808e-04 -2.0109117e-04]
  [-3.1536818e-04 -2.1053851e-04]
  [-3.2877922e-04 -2.1147728e-04]
  [-3.4058094e-04 -2.0717084e-04]
  [-3.4862757e-04 -1.9869208e-04]]], shape=(2, 5, 2), dtype=float32)
Num nodes per examples: 
tf.Tensor([678 678], shape=(2,), dtype=int32)


In the next, cells we are going to do experiment only over two particles for understanding the results

## Noise Injection - Random Walk
For each input state, particle and spatial dimension. It is performed the following steps

1. Accumulate across time
2. Random Walk
3. Pertud the stack velocities (updated the velocities)
4. Adjust the position features to mantained $\dot{p}^{t_k} = p^{t_k} - p^{t_{k-1}}$ for consistency

This method adds noise to each input, adding always noise to the previous state in sequence as in a random walk.

In this way, it seeks to simulate the accumulation of error in a rolllout.

Take in mind the concept of [random walk](https://en.wikipedia.org/wiki/Random_walk#:~:text=In%20mathematics%2C%20a%20random%20walk,space%20such%20as%20the%20integers.).
In mathematics, a random walk is a mathematical object, known as a stochastic or random process, that describes a path that consists of a succession of random steps on some mathematical space such as the integers.

In [19]:
# We want the noise scale in the velocity at the last step to be fixed.
# Because we are going to compose noise at each step using a random_walk:
# std_last_step**2 = num_velocities * std_each_step**2
# so to keep `std_last_step` fixed, we apply at each step:
# std_each_step `std_last_step / np.sqrt(num_input_velocities)`
# TODO(alvarosg): Make sure this is consistent with the value and
# description provided in the paper.
noise_std_last_step = 6.7e-4 # standard_noise # defined in FLAG arguments

num_velocities = velocity_sequence.shape.as_list()[1]
velocity_sequence_noise = tf.random.normal(
  tf.shape(velocity_sequence),
  stddev=noise_std_last_step / num_velocities ** 0.5,
  dtype=position_sequence.dtype)
print("velocity_sequence_noise: ")
print(velocity_sequence_noise)

print("---------------------")
# Apply the random walk. # cumulative sum # tf.cumsum([a, b, c])   # [a, a + b, a + b + c]
velocity_sequence_noise = tf.cumsum(velocity_sequence_noise, axis=1)
print("velocity_sequence_noise after random walk: ")
print(velocity_sequence_noise)

# Integrate the noise in the velocity to the positions, assuming
# an Euler intergrator and a dt = 1, and adding no noise to the very first
# position (since that will only be used to calculate the first position#
# change.
print("---------------------")
position_sequence_noise = tf.concat([
  tf.zeros_like(velocity_sequence_noise[:, 0:1]),
  tf.cumsum(velocity_sequence_noise, axis=1)], axis=1)
print("position sequence noise: ")
print(position_sequence_noise)

print("---------------------")
sampled_noise = position_sequence_noise

non_kinematic_mask = tf.logical_not(train.get_kinematic_mask(particle_type))
print("non_kinematic_mask: ")
print(non_kinematic_mask)

print("---------------------")
noise_mask = tf.cast(
    non_kinematic_mask, sampled_noise.dtype)[:, tf.newaxis, tf.newaxis]
print("noise_mask: ")
print(noise_mask)

print("---------------------")
print("Apply the mask to the noise") # just send to zero all the noise of a kinematic particle
sampled_noise *= noise_mask
print(sampled_noise)


velocity_sequence_noise: 
tf.Tensor(
[[[ 1.3896731e-04  2.9208095e-04]
  [-4.3777040e-05 -2.0929029e-04]
  [ 2.5434262e-04  6.6257853e-05]
  [-5.8383623e-04 -3.3508120e-05]
  [-3.7062203e-04  2.6564026e-05]]

 [[-9.1758535e-05  2.6945927e-04]
  [-3.9058787e-04 -9.2792856e-05]
  [ 2.8277788e-04  1.6330833e-04]
  [ 1.8657474e-05 -1.4110726e-04]
  [ 9.1974624e-04 -3.8296484e-06]]], shape=(2, 5, 2), dtype=float32)
---------------------
velocity_sequence_noise after random walk: 
tf.Tensor(
[[[ 1.38967313e-04  2.92080949e-04]
  [ 9.51902766e-05  8.27906624e-05]
  [ 3.49532900e-04  1.49048516e-04]
  [-2.34303327e-04  1.15540395e-04]
  [-6.04925328e-04  1.42104429e-04]]

 [[-9.17585348e-05  2.69459269e-04]
  [-4.82346397e-04  1.76666421e-04]
  [-1.99568516e-04  3.39974766e-04]
  [-1.80911040e-04  1.98867507e-04]
  [ 7.38835195e-04  1.95037865e-04]]], shape=(2, 5, 2), dtype=float32)
---------------------
position sequence noise: 
tf.Tensor(
[[[ 0.0000000e+00  0.0000000e+00]
  [ 1.3896731e-04  

In [24]:
n_particles_per_example[:-1]

<tf.Tensor: id=36782, shape=(1,), dtype=int32, numpy=array([678], dtype=int32)>

## Connectivity

Here, the author uses a k-d tree structure to deal with the points in n_dimension (max 3 dim for our proposes)

In computer science, a k-d tree (short for k-dimensional tree) is a space-partitioning data structure for organizing points in a k-dimensional space. k-d trees are a useful data structure for several applications, such as searches involving a multidimensional search key (e.g. range searches and nearest neighbor searches) and creating point clouds. k-d trees are a special case of binary space partitioning trees. For further explanation see: [wiki](https://en.wikipedia.org/wiki/K-d_tree)

In [33]:
import sonnet as snt
arr = tf.convert_to_tensor([[[1,2],[3,4],[5,6]],[[7,8],[9,10],[11,12]]])
print(arr)
print("----")
print(arr[0])
print("----")
snt.MergeDims(start=1, size=2)(
        arr)

tf.Tensor(
[[[ 1  2]
  [ 3  4]
  [ 5  6]]

 [[ 7  8]
  [ 9 10]
  [11 12]]], shape=(2, 3, 2), dtype=int32)
----
tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32)
----


<tf.Tensor: id=36809, shape=(2, 6), dtype=int32, numpy=
array([[ 1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12]], dtype=int32)>

In [44]:
position_sequence = tf.convert_to_tensor([[[1.,2.],[3.,4.],[5.,6.]],[[7.,8.],[9.,10.],[11.,12.]]])
most_recent_position = position_sequence[:, -1]

print(most_recent_position)

b = np.array([[4.,5.],[6.,7.]])

boundaries = tf.constant(b, dtype=tf.float32)
print("-----")
print(boundaries)
print("-----")
print(boundaries[:, 0])
print(tf.expand_dims(boundaries[:, 0], 0))
distance_to_lower_boundary = (
    most_recent_position - tf.expand_dims(boundaries[:, 0], 0))
print(distance_to_lower_boundary)
distance_to_upper_boundary = (
        tf.expand_dims(boundaries[:, 1], 0) - most_recent_position)
print("-----")
print(distance_to_upper_boundary)
tf.concat([distance_to_lower_boundary, distance_to_upper_boundary], axis=1)

tf.Tensor(
[[ 5.  6.]
 [11. 12.]], shape=(2, 2), dtype=float32)
-----
tf.Tensor(
[[4. 5.]
 [6. 7.]], shape=(2, 2), dtype=float32)
-----
tf.Tensor([4. 6.], shape=(2,), dtype=float32)
tf.Tensor([[4. 6.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[1. 0.]
 [7. 6.]], shape=(2, 2), dtype=float32)
-----
tf.Tensor(
[[ 0.  1.]
 [-6. -5.]], shape=(2, 2), dtype=float32)


<tf.Tensor: id=36967, shape=(2, 4), dtype=float32, numpy=
array([[ 1.,  0.,  0.,  1.],
       [ 7.,  6., -6., -5.]], dtype=float32)>